In [1]:
import sys
sys.path.append('../')

import tquant as tq
# from engine.globalconfig import GlobalConfig
# from engine.loader.curveload import MarketDataLoader

from datetime import date, timedelta
import tensorflow as tf
import pandas as pd
from datastore import *

import QuantLib as ql

In [2]:
tq.Settings.evaluation_date = date(2024, 4, 30)

calendar = tq.TARGET()
daycounter = tq.DayCounter(tq.DayCounterConvention.Actual360)
evaluation_date = tq.Settings.evaluation_date

In [3]:
rates = ir_eur_curve_estr['quote'].values
times = ir_eur_curve_estr['daycount'].values/365

estr_curve = tq.RateCurve(times, rates)

In [4]:
today = ql.Date(30,4,2024)
ql.Settings.instance().evaluationDate = today

sp = [1.0, 0.9941, 0.9826, 0.9674, 0.9488, 0.9246, 0.8945, 0.8645, 0.83484, 0.80614, 0.7784]
dp = [1 - s for s in sp]
ql_times = [today + int(np.round(day*365,0)) for day in times]
ql_times_sp = [today + int(n*365) for n in range(11)]
times_sp = [0 + int(n) for n in range(11)]

estr_curve = tq.RateCurve(times, rates)
default_curve = tq.SurvivalProbabilityCurve(times_sp, sp)

In [5]:
ql_crv = ql.SurvivalProbabilityCurve(ql_times_sp, sp, ql.Actual365Fixed(), ql.TARGET())
ql_crv.enableExtrapolation()

ql_estr_curve = ql.ZeroCurve(ql_times,rates,ql.Actual365Fixed())
ql_estr_handle = ql.RelinkableYieldTermStructureHandle()
ql_estr_handle.linkTo(ql_estr_curve)

In [6]:
probability = ql.DefaultProbabilityTermStructureHandle(ql_crv)
recoveryRate = 0.4
integralStep = ql.Period('1d')
engine = ql.IntegralCdsEngine(integralStep,probability, recoveryRate, ql_estr_handle)


In [7]:
spread = 34.6 / 10000
nominal = 1000000

cdsSchedule = ql.MakeSchedule(ql.Date(22,5,2024), 
                              ql.Date(22, 5, 2027), 
                              ql.Period('3M'),
                              ql.Quarterly, 
                              ql.TARGET(), 
                              ql.Following
                              )

cds = ql.CreditDefaultSwap(ql.Protection.Seller, nominal, spread, cdsSchedule, ql.Following, ql.Actual365Fixed())

cds.setPricingEngine(engine)

print(cds.NPV())
print(cds.couponLegNPV())
print(cds.defaultLegNPV())

-9224.361959851592
9726.17224526718
-18950.534205118773


In [8]:
# --------------
# Premium leg
start_date = date(2024, 5, 22)
fixed_schedule = [start_date]
fixed_schedule.extend([start_date + timedelta(365 *.25 *i) for i in range(1,13)])
fixed_spread = [34.6 / 10000]*len(fixed_schedule)
fixed_notional = [1000000]*len(fixed_schedule)
fixed_daycounter = tq.DayCounter(tq.DayCounterConvention.Actual365)
swap_type = tq.SwapType.Payer

premium_leg = tq.PremiumLeg(fixed_schedule, 
                            fixed_notional, 
                            fixed_spread, 
                            fixed_daycounter,
                            tq.CompoundingType.Simple, 
                            tq.Frequency.Quarterly
                            )
# --------------
# floating leg
default_schedule = [start_date]
default_schedule.extend([start_date + timedelta(365 *.25 *i) for i in range(1,13)])

floating_notional = [1000000]*len(default_schedule)
floating_daycounter = tq.DayCounter(tq.DayCounterConvention.Actual365)

default_leg = tq.DefaultLeg(default_schedule, 
                            floating_notional, 
                            recoveryRate, 
                            floating_daycounter,
                            tq.CompoundingType.Simple, 
                            tq.Frequency.Quarterly
                            )
# --------------
# cds object

cds = tq.CreditDefaultSwap.from_legs(swap_type, 
                           premium_leg,
                           default_leg)

cds_engine = tq.CDSAnalyticEngine(cds)

npv_cds = cds_engine.price(estr_curve, default_curve, evaluation_date)
print(npv_cds.numpy())

-9203.134535982168


In [9]:
print(fixed_schedule)

[datetime.date(2024, 5, 22), datetime.date(2024, 8, 21), datetime.date(2024, 11, 20), datetime.date(2025, 2, 19), datetime.date(2025, 5, 22), datetime.date(2025, 8, 21), datetime.date(2025, 11, 20), datetime.date(2026, 2, 19), datetime.date(2026, 5, 22), datetime.date(2026, 8, 21), datetime.date(2026, 11, 20), datetime.date(2027, 2, 19), datetime.date(2027, 5, 22)]


In [10]:
print(estr_curve.discount(1))
print(ql_estr_curve.discount(ql.Date(29,9,2024)))

tf.Tensor(0.9656633543206664, shape=(), dtype=float64)
0.9848361642803789


In [11]:
print(evaluation_date)
print(today)
print(default_curve.survival_probability(tq.date(2026,9,29), 
                                         tq.DayCounter(tq.DayCounterConvention.Actual365), 
                                         evaluation_date))
for p,d in zip (default_curve.survival_probabilities,default_curve.pillars):
    print(p,d)
print(ql_crv.survivalProbability(ql.Date(29,9,2026)))

2024-04-30
April 30th, 2024
tf.Tensor(0.9762413308352417, shape=(), dtype=float64)
tf.Tensor(1.0, shape=(), dtype=float64) 0
tf.Tensor(0.9941, shape=(), dtype=float64) 1
tf.Tensor(0.9826, shape=(), dtype=float64) 2
tf.Tensor(0.9674, shape=(), dtype=float64) 3
tf.Tensor(0.9488, shape=(), dtype=float64) 4
tf.Tensor(0.9246, shape=(), dtype=float64) 5
tf.Tensor(0.8945, shape=(), dtype=float64) 6
tf.Tensor(0.8645, shape=(), dtype=float64) 7
tf.Tensor(0.83484, shape=(), dtype=float64) 8
tf.Tensor(0.80614, shape=(), dtype=float64) 9
tf.Tensor(0.7784, shape=(), dtype=float64) 10
0.9762701369863014


In [12]:
default_leg.display_flows()

,start_period,end_period,payment_date,notional,recovery,accrual,day_counter
0,2024-05-22,2024-05-23,2024-05-23,1000000,0.4,0.00274,Actual365
0,2024-05-23,2024-05-24,2024-05-24,1000000,0.4,0.00274,Actual365
0,2024-05-24,2024-05-25,2024-05-25,1000000,0.4,0.00274,Actual365
0,2024-05-25,2024-05-26,2024-05-26,1000000,0.4,0.00274,Actual365
0,2024-05-26,2024-05-27,2024-05-27,1000000,0.4,0.00274,Actual365
...,...,...,...,...,...,...,...
0,2027-05-17,2027-05-18,2027-05-18,1000000,0.4,0.00274,Actual365
0,2027-05-18,2027-05-19,2027-05-19,1000000,0.4,0.00274,Actual365
0,2027-05-19,2027-05-20,2027-05-20,1000000,0.4,0.00274,Actual365
0,2027-05-20,2027-05-21,2027-05-21,1000000,0.4,0.00274,Actual365
